In [2]:
!pip install praw
!pip install psaw

import praw
import pandas as pd
import datetime
import numpy as np
import string 
import re
from psaw import PushshiftAPI
import datetime as dt

     |████████████████████████████████| 176 kB 27.1 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 


In [3]:
reddit = praw.Reddit(client_id='gjin2lXgM8CYw6rjuNIfJA', client_secret='Vkof6BhymxjJrlNldl3NvQo3yYZFDA', user_agent='bdma')
api = PushshiftAPI(reddit)

In [4]:
start_epoch=int(dt.datetime(2020, 1, 1).timestamp())
gen = api.search_submissions(after=start_epoch,
                            subreddit='depression',
                            filter=['title'],
                            limit=2000)
tweets = []
for submissions in gen:
  tweets.append(submissions.title)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bot

In [5]:
posts = pd.DataFrame(tweets, columns=['clean_comment'])

In [6]:
print("shape :", posts.shape)
posts.head()

shape : (2000, 1)


,clean_comment
0,I'm so sick of this shit
1,It will pass...
2,Need some help
3,I need a reason
4,I've hit rock bottom. :/


In [7]:
def processTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', str(tweet))
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + string.punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip('') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uffff') 
    emoticons =  ('😇','😊','❤️','😘','💞','💖','🤗','💕','👏','🎉','👍',
              '😂','😡','😠','😭','🤦‍','🤷🏼‍','😞','👎','😱','😓','🔝')
    tweet = re.sub('[^ a-zA-Zà-ú'
                            '\😇\😊\❤️\😘\💞\💖\🤗\💕\👏\🎉\👍'
                            '\😂\😡\😠\😭\🤦‍\🤷🏼‍\😞\😱\👎\😓\🔝]', " ",tweet)    
    for word in emoticons:
        tweet = re.sub(word, " "+word+" ",tweet) 

    return tweet

posts['clean_comment'] = posts['clean_comment'].apply(processTweet)

In [8]:
from textblob import TextBlob
import matplotlib.pyplot as plt

In [9]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    
    if analysis.sentiment.polarity > 0:
        return "happy"
    else:
        return "depressed"

    
posts['category'] = posts['clean_comment'].apply(analyze_sentiment)
posts.head()

,clean_comment,category
0,sick this shit,depressed
1,will pass,depressed
2,need some help,depressed
3,need reason,depressed
4,hit rock bottom,depressed


In [10]:
positives = posts['category'][posts.category == "happy"]
negatives = posts['category'][posts.category == "depressed"]



print('number of positve categorized text is:  {}'.format(len(positives)))
print('number of negative categorized text is: {}'.format(len(negatives)))
print('total length of the data is:            {}'.format(posts.shape[0]))

number of positve categorized text is:  362
number of negative categorized text is: 1638
total length of the data is:            2000


In [11]:
df1 = posts.filter(['clean_comment','category'], axis=1)
df1.head()

,clean_comment,category
0,sick this shit,depressed
1,will pass,depressed
2,need some help,depressed
3,need reason,depressed
4,hit rock bottom,depressed


In [12]:
df1.to_csv('/content/sample_data/df1.csv', index = False)

In [13]:
gen2 = api.search_submissions(after=start_epoch,
                            subreddit='happy',
                            filter=['title'],
                            limit=2000)
tweets2 = []
for submissions in gen2:
  tweets2.append(submissions.title)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bot

In [14]:
posts2 = pd.DataFrame(tweets2, columns=['clean_comment'])

In [15]:
print("shape :", posts2.shape)
posts2.head()

shape : (2000, 1)


,clean_comment
0,how to get over a breakup properly
1,Just a small piece of happiness
2,In 2019 My wife and I had our dream Harry Pott...
3,It is 11:00pm and I’ve just finished my Masters
4,"20 years later, my brother and I coworkers at ..."


In [16]:
posts2['clean_comment'] = posts2['clean_comment'].apply(processTweet)

In [17]:
posts2['category'] = posts2['clean_comment'].apply(analyze_sentiment)
posts2.head()

,clean_comment,category
0,how get over breakup properly,depressed
1,just small piece happiness,happy
2,wife and had our dream harry potter wedd...,happy
3,pm and just finished masters,depressed
4,years later brother and coworkers the same co...,depressed


In [18]:
positives = posts2['category'][posts2.category == "happy"]
negatives = posts2['category'][posts2.category == "depressed"]



print('number of positve categorized text is:  {}'.format(len(positives)))
print('number of negative categorized text is: {}'.format(len(negatives)))
print('total length of the data is:            {}'.format(posts.shape[0]))

number of positve categorized text is:  1273
number of negative categorized text is: 727
total length of the data is:            2000


In [19]:
df2 = posts2.filter(['clean_comment','category'], axis=1)
df2.head()

,clean_comment,category
0,how get over breakup properly,depressed
1,just small piece happiness,happy
2,wife and had our dream harry potter wedd...,happy
3,pm and just finished masters,depressed
4,years later brother and coworkers the same co...,depressed


In [20]:
df2.to_csv('/content/sample_data/df2.csv', index = False)

In [21]:
gen3 = api.search_submissions(after=start_epoch,
                            subreddit='angry',
                            filter=['title'],
                            limit=2000)

In [22]:
tweets3 = []
for submissions in gen3:
  tweets3.append(submissions.title)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bot

In [23]:
posts3 = pd.DataFrame(tweets3, columns=['clean_comment'])

In [24]:
print("shape :", posts3.shape)
posts3.head()

shape : (2000, 1)


,clean_comment
0,Why do cafes close SO fucking EARLY?
1,Call Your Mother
2,I don’t understand why some people commit suic...
3,i just want someone to fucking care
4,F You.


In [25]:
posts3['clean_comment'] = posts3['clean_comment'].apply(processTweet)

In [64]:
def analyze_sentiment1(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity >= 0:
        return "calm"
    else:
        return "angry"

posts3['category'] = posts3['clean_comment'].apply(analyze_sentiment1)
posts3.head()

,clean_comment,category
0,why cafes close fucking early,calm
1,call your mother,calm
2,don understand why some people commit suicid...,calm
3,just want someone fucking care,angry
4,you,calm


In [65]:
positives = posts3['category'][posts3.category == "calm"]
negatives = posts3['category'][posts3.category == "angry"]


print('number of positve categorized text is:  {}'.format(len(positives)))
print('number of negative categorized text is: {}'.format(len(negatives)))
print('total length of the data is:            {}'.format(posts.shape[0]))

number of positve categorized text is:  973
number of negative categorized text is: 1027
total length of the data is:            2000


In [66]:
df3 = posts3.filter(['clean_comment','category'], axis=1)
df3.head()

,clean_comment,category
0,why cafes close fucking early,calm
1,call your mother,calm
2,don understand why some people commit suicid...,calm
3,just want someone fucking care,angry
4,you,calm


In [67]:
df3.to_csv('/content/sample_data/df3.csv', index = False)

In [68]:
gen4 = api.search_submissions(after=start_epoch,
                            subreddit='calm',
                            filter=['title'],
                            limit=2000)

In [69]:
tweets4 = []
for submissions in gen4:
  tweets4.append(submissions.title)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bot

In [70]:
posts4 = pd.DataFrame(tweets4, columns=['clean_comment'])

In [71]:
print("shape :", posts4.shape)
posts4.head()

shape : (1139, 1)


,clean_comment
0,WONDERFUL STREAM SOUND | THE SOUND OF WATER FO...
1,The Wine Fields In Autumn On A Sunny Day ASMR ...
2,A Perfect new pieces of Relaxation music that ...
3,The time to relax is when you don't have time ...
4,"""The time to relax is when you don't have time..."


In [72]:
posts4['clean_comment'] = posts4['clean_comment'].apply(processTweet)

In [73]:
posts4['category'] = posts4['clean_comment'].apply(analyze_sentiment1)
posts4.head()

,clean_comment,category
0,wonderful stream sound the sound water for pea...,calm
1,the wine fields autumn sunny day asmr walk,calm
2,perfect new pieces relaxation music that love...,calm
3,the time relax when you don have time for,calm
4,the time relax when you don have time for syd...,calm


In [74]:
positives = posts4['category'][posts4.category == "calm"]
negatives = posts4['category'][posts4.category == "angry"]


print('number of positve categorized text is:  {}'.format(len(positives)))
print('number of negative categorized text is: {}'.format(len(negatives)))
print('total length of the data is:            {}'.format(posts.shape[0]))

number of positve categorized text is:  1044
number of negative categorized text is: 95
total length of the data is:            2000


In [75]:
df4 = posts4.filter(['clean_comment','category'], axis=1)
df4.head()

,clean_comment,category
0,wonderful stream sound the sound water for pea...,calm
1,the wine fields autumn sunny day asmr walk,calm
2,perfect new pieces relaxation music that love...,calm
3,the time relax when you don have time for,calm
4,the time relax when you don have time for syd...,calm


In [76]:
df4.to_csv('/content/sample_data/df4.csv', index = False)

In [77]:
a = pd.read_csv("/content/drive/MyDrive/df1.csv")
b = pd.read_csv("/content/drive/MyDrive/df2.csv")
c = pd.concat([a,b])

In [79]:
d = pd.read_csv("/content/drive/MyDrive/df3.csv")
e = pd.read_csv("/content/drive/MyDrive/df4.csv")
f = pd.concat([d,e])

In [82]:
g = pd.concat([c,f])
g.head()

,clean_comment,category
0,sick this shit,depressed
1,will pass,depressed
2,need some help,depressed
3,need reason,depressed
4,hit rock bottom,depressed


In [83]:
print("Shape :", a.shape)
print("Shape :", b.shape)
print("Shape :", c.shape)
print("Shape :", d.shape)
print("Shape :", e.shape)
print("Shape :", f.shape)
print("Shape :", g.shape)

Shape : (2000, 2)
Shape : (2000, 2)
Shape : (4000, 2)
Shape : (2000, 2)
Shape : (1139, 2)
Shape : (3139, 2)
Shape : (7139, 2)


In [85]:
g.to_csv("/content/drive/MyDrive/new_moods.csv", index = False)

In [84]:
g['category'].value_counts()

depressed    2365
calm         2017
happy        1635
angry        1122
Name: category, dtype: int64